# Methods to be tested in this NLP model

1. Bag of words 
2. Tf idf 
3. word2vec 
4. Glove 
5. Bert 
6. Sbert etc... 
7. 4.5 sent2vec

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('../data/potential-talents - Aspiring human resources - seeking human resources.csv')

In [4]:
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


### Using a simple function to find keywords:

In [9]:
# First option
def predict_fit(job_title):
    keywords = ["aspiring human resources", "seeking human resources"]
    for keyword in keywords:
        if keyword in job_title.lower():
            return 1  # Fit
    return 0  # Not fit

In [10]:
df['fit'] = df['job_title'].apply(predict_fit)

In [11]:
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,1
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,0
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,1
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,0
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0


### Using Bag of Words:

In [ ]:
df = pd.read_csv('../data/potential-talents - Aspiring human resources - seeking human resources.csv')

In [5]:
texts = df['job_title']

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

In [7]:
feature_names = vectorizer.get_feature_names_out()
df_bow = pd.DataFrame(X.toarray(), columns=feature_names)

In [14]:
merged_df = pd.concat([df, df_bow], axis=1)

In [15]:
merged_df.head()

,id,job_title,location,connection,fit,2019,2020,2621,408,709,...,up,victoria,wellington,western,westfield,with,within,woodlands,work,world
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
